In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
import re

import h5py
from tqdm import tqdm

from utils import load_and_format_data_as_df, get_birds_in_time_range

In [ ]:
# # this will load the data from the hdf5 file and format it as a pandas dataframe
# # the hdf5 files need to be in the current directory (same as the notebook)
# # ONLY RUN THIS CELL ONCE TO GET THE CSV FILES

# for dirname, _, filenames in os.walk('./'):
#     for filename in filenames:
#         path = os.path.join(dirname, filename)
#         if '.hdf5' in filename:
#             print(f'loading {path}')
#             df = load_and_format_data_as_df(path)
#             df.to_csv(f"./{filename.replace('.hdf5', '.csv')}", index=False)
#             print(f'saved under {filename.replace(".hdf5", ".csv")}')

In [2]:
df_A = pd.read_csv('A_nodes.csv', index_col=0)
df_B = pd.read_csv('B_nodes.csv', index_col=0)
df_C = pd.read_csv('C_nodes.csv', index_col=0)

In [ ]:
# data encompasses "flock formation, circling, and landing" over a 30 minute period
# ~1800 chimney swift birds
# there are regions reported where the birds are highly correlated - when flying - and less correlated - when landing - as the intention of the study

In [11]:
# range of time in minutes
(df_A.time.max() - df_A.time.min()) / 60, (df_B.time.max() - df_B.time.min()) / 60, (df_C.time.max() - df_C.time.min()) / 60

(8.505727950172396, 9.431653876098322, 8.19263708152597)

In [20]:
# individual time ranges for tracks
# A
df_A.groupby('tid').apply(lambda x: x.time.max() - x.time.min()).describe()

count    24966.000000
mean         7.405351
std          6.848828
min          0.934268
25%          2.535869
50%          5.071738
75%         10.010010
max        100.066733
dtype: float64

In [21]:
# individual time ranges for tracks
# B
df_B.groupby('tid').apply(lambda x: x.time.max() - x.time.min()).describe()

count    78498.000000
mean         6.520097
std          5.619826
min          0.934268
25%          2.435769
50%          4.671338
75%          8.842176
max         59.092426
dtype: float64

In [22]:
# individual time ranges for tracks
# C
df_C.groupby('tid').apply(lambda x: x.time.max() - x.time.min()).describe()

count    39403.000000
mean         4.478242
std          3.859843
min          0.934268
25%          1.968635
50%          3.169837
75%          5.472139
max         42.442442
dtype: float64

In [103]:
# get all tracks that last for the entire duration

df = df_A.copy()

time_start = 40
time_end = 45




get_birds_in_time_range(df, time_start, time_end)

,V,hdg,hdg_rate,tid,vx,vy,vz,x,y,z,time
640622,3.363078,141.568420,-4.837680,740.0,-2.616330,2.076026,-0.393989,23.094999,53.719002,48.104000,40.006673
640623,4.791535,184.965073,9.016342,741.0,-4.667496,-0.405486,-1.004425,-30.546000,35.678001,24.851000,40.006673
640624,5.903653,201.401077,17.967407,746.0,-5.465564,-2.142050,-0.626382,-43.201000,57.820000,26.136999,40.006673
640625,4.376662,146.325424,11.189796,747.0,-3.641464,2.426221,-0.091429,-44.298000,21.427999,43.200001,40.006673
640626,4.325809,107.295685,21.896231,748.0,-1.285429,4.128131,-0.137233,-28.393999,48.778999,33.161999,40.006673
...,...,...,...,...,...,...,...,...,...,...,...
1175574,4.335289,153.155807,-18.847616,1110.0,-3.856245,1.951665,-0.339273,-16.099001,15.743000,40.852001,44.978312
1175586,5.836926,209.760086,-11.698069,1289.0,-5.066691,-2.897037,0.074384,20.823000,44.570999,25.780001,44.978312
1175588,6.206317,178.804810,-30.006010,1391.0,-6.203670,0.129427,0.126908,4.980200,16.142000,27.521000,44.978312
1175589,4.897807,150.849640,-39.372532,1404.0,-4.243038,2.366531,-0.620221,-36.894001,0.248730,28.547001,44.978312


In [65]:
# calculate velocity order parameter

